# Connection semantics prototype

## Motivation and concepts
Typically one would create connections in NEST by calling `Connect()` on two NodeCollections. However, every time `Connect()` is called, it has to go from PyNEST, through SLI and into the C++ kernel, iterate over connections, before it returns to PyNEST. One may also use multiple threads in the iteration over connections, where initiating and ending a parallel region causes an additional overhead. Therefore, if a simulation involves many `Connect()` calls, this entails a significant overhead in the connection stage.

To reduce the overhead of repeated calls to `Connect()`, we introduce projections. With projections, instead of immediately creating the connections, the connection specifications are stored until the user specifies that the connections should be made, typically after specifying all projections. That way we can have only a single push from PyNEST through SLI to the C++ kernel, and reduce the number of times we enter the parallel region.

## How it works
The projections are introduced as objects based on existing connection rules, such as `OneToOne` from the `one_to_one` rule, or `FixedIndegree` from the `fixed_indegree` rule. The projection object is then added to a connection buffer on the PyNEST level, using `Connect()`. When all projections have been defined and added to the buffer, the user can call `BuildNetwork()`, which sends all projections in a single package to the C++ level. Once on the C++ level, the projections that have the same connection rule can be combined automatically, reducing the number of times we enter the parallel region when connecting.

## What is currently implemented

- A PyNEST interface with several projection classes, implemented in a submodule `nest.projections`.
- Projections work with both normal connections, and spatially structured connections involving for example distance based probabilities or masks.
- Classes for some of the synapse models, in the submodule `nest.synapsemodels`. The synapse object created can be used as the synapse specification when creating projections.
- Synapse objects can be combined with `CollocatedSynapses`, and used instead of a single synapse model object when creating a projection.
- Basic C++ implementation for connection, without any processing of the projections before connecting.

In [1]:
import nest

## Projection object

Introducing the Projection object, representing projection between two populations with a certain rule. The projection object is added to a buffer with `projections.Connect()`, and held until calling `projections.BuildNetwork()`.

### `OneToOne`

In [2]:
nest.ResetKernel()
n = nest.Create('iaf_psc_alpha', 5)

projection = nest.projections.OneToOne(source=n, target=n)
nest.projections.Connect(projection)
nest.projections.BuildNetwork()

conns = nest.GetConnections()
print(conns)

 source   target   synapse model   weight   delay 
-------- -------- --------------- -------- -------
      1        1  static_synapse    1.000   1.000
      2        2  static_synapse    1.000   1.000
      3        3  static_synapse    1.000   1.000
      4        4  static_synapse    1.000   1.000
      5        5  static_synapse    1.000   1.000


### `FixedIndegree`
Different Projections can take different arguments, depending on the connection rule.

In [3]:
nest.ResetKernel()
n = nest.Create('iaf_psc_alpha', 5)

projection = nest.projections.FixedIndegree(source=n, target=n, indegree=5)
nest.projections.Connect(projection)
nest.projections.BuildNetwork()

conns = nest.GetConnections()
print(conns)

 source   target   synapse model   weight   delay 
-------- -------- --------------- -------- -------
      1        3  static_synapse    1.000   1.000
      1        1  static_synapse    1.000   1.000
      1        3  static_synapse    1.000   1.000
      2        1  static_synapse    1.000   1.000
      2        2  static_synapse    1.000   1.000
      2        2  static_synapse    1.000   1.000
      2        2  static_synapse    1.000   1.000
      2        2  static_synapse    1.000   1.000
      2        3  static_synapse    1.000   1.000
      2        1  static_synapse    1.000   1.000
      2        4  static_synapse    1.000   1.000
      2        4  static_synapse    1.000   1.000
      2        5  static_synapse    1.000   1.000
      3        3  static_synapse    1.000   1.000
      3        5  static_synapse    1.000   1.000
      3        5  static_synapse    1.000   1.000
      3        5  static_synapse    1.000   1.000
      4        1  static_synapse    1.000   1.00

### `FixedOutdegree`

In [4]:
nest.ResetKernel()
n = nest.Create('iaf_psc_alpha', 5)

projection = nest.projections.FixedOutdegree(source=n, target=n, outdegree=5)
nest.projections.Connect(projection)
nest.projections.BuildNetwork()

conns = nest.GetConnections()
print(conns)

 source   target   synapse model   weight   delay 
-------- -------- --------------- -------- -------
      1        3  static_synapse    1.000   1.000
      1        2  static_synapse    1.000   1.000
      1        4  static_synapse    1.000   1.000
      1        2  static_synapse    1.000   1.000
      1        1  static_synapse    1.000   1.000
      2        1  static_synapse    1.000   1.000
      2        3  static_synapse    1.000   1.000
      2        5  static_synapse    1.000   1.000
      2        4  static_synapse    1.000   1.000
      2        3  static_synapse    1.000   1.000
      3        4  static_synapse    1.000   1.000
      3        5  static_synapse    1.000   1.000
      3        5  static_synapse    1.000   1.000
      3        5  static_synapse    1.000   1.000
      3        2  static_synapse    1.000   1.000
      4        2  static_synapse    1.000   1.000
      4        2  static_synapse    1.000   1.000
      4        4  static_synapse    1.000   1.00

### `PairwiseBernoulli`
Projections can store synapse specifications.

In [5]:
nest.ResetKernel()
n = nest.Create('iaf_psc_alpha', 10)

projection = nest.projections.PairwiseBernoulli(source=n, target=n, 
                                                p=0.2)
nest.projections.Connect(projection)
nest.projections.BuildNetwork()

conns = nest.GetConnections()
print(conns)

 source   target   synapse model   weight   delay 
-------- -------- --------------- -------- -------
      1        2  static_synapse    1.000   1.000
      1        4  static_synapse    1.000   1.000
      1        5  static_synapse    1.000   1.000
      1        9  static_synapse    1.000   1.000
      1       10  static_synapse    1.000   1.000
      2        1  static_synapse    1.000   1.000
      2        7  static_synapse    1.000   1.000
      2        8  static_synapse    1.000   1.000
      3        2  static_synapse    1.000   1.000
      3        5  static_synapse    1.000   1.000
      3        8  static_synapse    1.000   1.000
      4        4  static_synapse    1.000   1.000
      5        5  static_synapse    1.000   1.000
      6        3  static_synapse    1.000   1.000
      6        5  static_synapse    1.000   1.000
      7        6  static_synapse    1.000   1.000
      8        3  static_synapse    1.000   1.000
      8        4  static_synapse    1.000   1.00

### `FixedTotalNumber`
Can also pass arguments that controls if autapses or multapses are possible.

In [6]:
nest.ResetKernel()
n = nest.Create('iaf_psc_alpha', 5)

projection = nest.projections.FixedTotalNumber(source=n, target=n, N=10, 
                                               allow_autapses=True,
                                               allow_multapses=True)
nest.projections.Connect(projection)
nest.projections.BuildNetwork()

conns = nest.GetConnections()
print(conns)

 source   target   synapse model   weight   delay 
-------- -------- --------------- -------- -------
      1        2  static_synapse    1.000   1.000
      1        4  static_synapse    1.000   1.000
      2        1  static_synapse    1.000   1.000
      2        5  static_synapse    1.000   1.000
      2        2  static_synapse    1.000   1.000
      2        5  static_synapse    1.000   1.000
      2        4  static_synapse    1.000   1.000
      3        4  static_synapse    1.000   1.000
      4        2  static_synapse    1.000   1.000
      4        2  static_synapse    1.000   1.000


## Multiple projections
Multiple Projections can be added to the buffer before connecting by calling `projections.Connect()` multiple times.

In [7]:
nest.ResetKernel()

N = 10
IN_A = 2
IN_B = 5
n = nest.Create('iaf_psc_alpha', N)
nest.projections.Connect(nest.projections.FixedIndegree(source=n, target=n, indegree=IN_A))
nest.projections.Connect(nest.projections.FixedIndegree(source=n, target=n, indegree=IN_B))
nest.projections.BuildNetwork()

conns = nest.GetConnections()
print(conns)

 source   target   synapse model   weight   delay 
-------- -------- --------------- -------- -------
      1        9  static_synapse    1.000   1.000
      1        4  static_synapse    1.000   1.000
      1        3  static_synapse    1.000   1.000
      1        6  static_synapse    1.000   1.000
      1        6  static_synapse    1.000   1.000
      1        3  static_synapse    1.000   1.000
      1        7  static_synapse    1.000   1.000
      1        2  static_synapse    1.000   1.000
      1        8  static_synapse    1.000   1.000
      2        6  static_synapse    1.000   1.000
      2        1  static_synapse    1.000   1.000
      2        5  static_synapse    1.000   1.000
      2        5  static_synapse    1.000   1.000
      2        5  static_synapse    1.000   1.000
      2        8  static_synapse    1.000   1.000
     ⋮        ⋮               ⋮        ⋮       ⋮ 
      7        9  static_synapse    1.000   1.000
      8       10  static_synapse    1.000   1.00

Or by passing a list of Projections.

In [8]:
nest.ResetKernel()

N = 10
IN_A = 2
IN_B = 5
n = nest.Create('iaf_psc_alpha', N)
projections = [nest.projections.FixedIndegree(source=n, target=n, indegree=IN_A),
               nest.projections.FixedIndegree(source=n, target=n, indegree=IN_B)]
nest.projections.Connect(projections)
nest.projections.BuildNetwork()

conns = nest.GetConnections()
print(conns)

 source   target   synapse model   weight   delay 
-------- -------- --------------- -------- -------
      1        9  static_synapse    1.000   1.000
      1        4  static_synapse    1.000   1.000
      1        3  static_synapse    1.000   1.000
      1        6  static_synapse    1.000   1.000
      1        6  static_synapse    1.000   1.000
      1        3  static_synapse    1.000   1.000
      1        7  static_synapse    1.000   1.000
      1        2  static_synapse    1.000   1.000
      1        8  static_synapse    1.000   1.000
      2        6  static_synapse    1.000   1.000
      2        1  static_synapse    1.000   1.000
      2        5  static_synapse    1.000   1.000
      2        5  static_synapse    1.000   1.000
      2        5  static_synapse    1.000   1.000
      2        8  static_synapse    1.000   1.000
     ⋮        ⋮               ⋮        ⋮       ⋮ 
      7        9  static_synapse    1.000   1.000
      8       10  static_synapse    1.000   1.00

## Using `ConnectImmediately()`
Using `projections.ConnectImmediately()` will immediately construct the connections of a Projection.

In [9]:
nest.ResetKernel()

n = nest.Create('iaf_psc_alpha', 5)
nest.projections.ConnectImmediately(nest.projections.OneToOne(source=n, target=n))

conns = nest.GetConnections()
print(conns)


 source   target   synapse model   weight   delay 
-------- -------- --------------- -------- -------
      1        1  static_synapse    1.000   1.000
      2        2  static_synapse    1.000   1.000
      3        3  static_synapse    1.000   1.000
      4        4  static_synapse    1.000   1.000
      5        5  static_synapse    1.000   1.000


Or for all Projections in a list.

In [10]:
nest.ResetKernel()

N = 5
IN_A = 2
IN_B = 4
n = nest.Create('iaf_psc_alpha', N)

projections = [nest.projections.FixedIndegree(source=n, target=n, indegree=IN_A),
               nest.projections.FixedIndegree(source=n, target=n, indegree=IN_B)]
nest.projections.ConnectImmediately(projections)

conns = nest.GetConnections()
print(conns)

 source   target   synapse model   weight   delay 
-------- -------- --------------- -------- -------
      1        1  static_synapse    1.000   1.000
      1        1  static_synapse    1.000   1.000
      1        5  static_synapse    1.000   1.000
      1        4  static_synapse    1.000   1.000
      1        1  static_synapse    1.000   1.000
      2        5  static_synapse    1.000   1.000
      2        4  static_synapse    1.000   1.000
      2        4  static_synapse    1.000   1.000
      2        5  static_synapse    1.000   1.000
      2        5  static_synapse    1.000   1.000
      2        3  static_synapse    1.000   1.000
      2        1  static_synapse    1.000   1.000
      2        5  static_synapse    1.000   1.000
      2        2  static_synapse    1.000   1.000
      2        1  static_synapse    1.000   1.000
      2        2  static_synapse    1.000   1.000
      2        5  static_synapse    1.000   1.000
      2        3  static_synapse    1.000   1.00

## Synapse model object
### `static_synapse`

In [11]:
nest.ResetKernel()
n = nest.Create('iaf_psc_alpha', 5)

synapse = nest.synapsemodels.static(weight=0.5, delay=0.7)

projection = nest.projections.OneToOne(source=n, target=n, syn_spec=synapse)
nest.projections.Connect(projection)
nest.projections.BuildNetwork()

conns = nest.GetConnections()
print(conns)

 source   target   synapse model   weight   delay 
-------- -------- --------------- -------- -------
      1        1  static_synapse   0.5000  0.7000
      2        2  static_synapse   0.5000  0.7000
      3        3  static_synapse   0.5000  0.7000
      4        4  static_synapse   0.5000  0.7000
      5        5  static_synapse   0.5000  0.7000


### `stdp_synapse`

In [12]:
nest.ResetKernel()
n = nest.Create('iaf_psc_alpha', 5)

synapse = nest.synapsemodels.stdp(weight=nest.random.uniform(0.5, 1.0), tau_plus=17.)

projection = nest.projections.OneToOne(source=n, target=n, syn_spec=synapse)
nest.projections.Connect(projection)
nest.projections.BuildNetwork()

conns = nest.GetConnections()
print(conns)
print('tau_plus =', conns.tau_plus)

 source   target   synapse model   weight   delay 
-------- -------- --------------- -------- -------
      1        1    stdp_synapse   0.6402   1.000
      2        2    stdp_synapse   0.5563   1.000
      3        3    stdp_synapse   0.6320   1.000
      4        4    stdp_synapse   0.9225   1.000
      5        5    stdp_synapse   0.8900   1.000
tau_plus = [17.0, 17.0, 17.0, 17.0, 17.0]


## Usage examples

### Generate projections iteratively, with duplicated projections
By cloning a Projection, one can iteratively generate similar projections, where only a few parameters are changing.

In [13]:
nest.ResetKernel()
n1 = nest.Create('iaf_psc_alpha', 3)
n2 = nest.Create('iaf_psc_exp', 3)

synapse = nest.synapsemodels.static(weight=0.5, delay=0.7)
base_projection = nest.projections.FixedTotalNumber(source=n1, target=n2,
                                                    N=5,
                                                    syn_spec=synapse)
for s, t in ((n1, n2), (n1, n1), (n2, n2), (n2, n1)):
    p = base_projection.clone()  # Create a copy of the base Projection.
    p.source = s  # Change some parameters
    p.target = t  # of the copied Projection.
    nest.projections.Connect(p)  # Add the copy to the buffer.
nest.projections.BuildNetwork()

conns = nest.GetConnections()
print(conns)
print('Number of connections:', len(conns))

 source   target   synapse model   weight   delay 
-------- -------- --------------- -------- -------
      1        4  static_synapse   0.5000  0.7000
      1        6  static_synapse   0.5000  0.7000
      1        4  static_synapse   0.5000  0.7000
      1        1  static_synapse   0.5000  0.7000
      1        2  static_synapse   0.5000  0.7000
      1        3  static_synapse   0.5000  0.7000
      2        2  static_synapse   0.5000  0.7000
      2        3  static_synapse   0.5000  0.7000
      3        4  static_synapse   0.5000  0.7000
      3        4  static_synapse   0.5000  0.7000
      4        4  static_synapse   0.5000  0.7000
      4        4  static_synapse   0.5000  0.7000
      4        3  static_synapse   0.5000  0.7000
      4        3  static_synapse   0.5000  0.7000
      4        1  static_synapse   0.5000  0.7000
      5        5  static_synapse   0.5000  0.7000
      5        4  static_synapse   0.5000  0.7000
      5        1  static_synapse   0.5000  0.700

### Using `CollocatedSynapses` in projections

In [14]:
nest.ResetKernel()
n = nest.Create('iaf_psc_alpha', 2)

syn_spec = nest.CollocatedSynapses(nest.synapsemodels.static(weight=-2.),
                                   nest.synapsemodels.stdp(weight=3.))

projection = nest.projections.FixedIndegree(source=n, target=n, indegree=3, syn_spec=syn_spec)
nest.projections.Connect(projection)
nest.projections.BuildNetwork()

conns = nest.GetConnections()
print(conns)

 source   target   synapse model   weight   delay 
-------- -------- --------------- -------- -------
      1        1  static_synapse   -2.000   1.000
      1        1  static_synapse   -2.000   1.000
      1        1  static_synapse   -2.000   1.000
      1        2  static_synapse   -2.000   1.000
      2        2  static_synapse   -2.000   1.000
      2        2  static_synapse   -2.000   1.000
      1        1    stdp_synapse    3.000   1.000
      1        1    stdp_synapse    3.000   1.000
      1        1    stdp_synapse    3.000   1.000
      1        2    stdp_synapse    3.000   1.000
      2        2    stdp_synapse    3.000   1.000
      2        2    stdp_synapse    3.000   1.000


### Spatial connections

In [15]:
nest.ResetKernel()

dim = [4, 5]
extent = [10., 10.]
layer = nest.Create('iaf_psc_alpha', positions=nest.spatial.grid(dim, extent=extent))

mask = {'rectangular': {
        'lower_left': [-5., -5.],
        'upper_right': [0., 0.]}}
projection = nest.projections.FixedIndegree(source=layer, target=layer, indegree=1, mask=mask)

nest.projections.Connect(projection)
nest.projections.BuildNetwork()

conns = nest.GetConnections()
print(conns)

 source   target   synapse model   weight   delay 
-------- -------- --------------- -------- -------
      1        1  static_synapse    1.000   1.000
      1       11  static_synapse    1.000   1.000
      2        2  static_synapse    1.000   1.000
      2        6  static_synapse    1.000   1.000
      3        3  static_synapse    1.000   1.000
      3       12  static_synapse    1.000   1.000
      3       13  static_synapse    1.000   1.000
      4        8  static_synapse    1.000   1.000
      5        4  static_synapse    1.000   1.000
      5        5  static_synapse    1.000   1.000
      5        9  static_synapse    1.000   1.000
      5       10  static_synapse    1.000   1.000
      8        7  static_synapse    1.000   1.000
     10       14  static_synapse    1.000   1.000
     10       15  static_synapse    1.000   1.000
     10       19  static_synapse    1.000   1.000
     12       17  static_synapse    1.000   1.000
     13       16  static_synapse    1.000   1.00

### Spatial connections with `CollocatedSynapses`

In [16]:
nest.ResetKernel()

dim = [4, 2]
extent = [10., 10.]
layer = nest.Create('iaf_psc_alpha', positions=nest.spatial.grid(dim, extent=extent))

mask = {'rectangular': {
        'lower_left': [-5., -5.],
        'upper_right': [0., 0.]}}

syn_spec = nest.CollocatedSynapses(nest.synapsemodels.static(weight=-2.),
                                   nest.synapsemodels.stdp(weight=3.))

projection = nest.projections.FixedIndegree(source=layer, target=layer, indegree=1, syn_spec=syn_spec, mask=mask)

nest.projections.Connect(projection)
nest.projections.BuildNetwork()

conns = nest.GetConnections()
print(conns)

 source   target   synapse model   weight   delay 
-------- -------- --------------- -------- -------
      1        1  static_synapse   -2.000   1.000
      2        2  static_synapse   -2.000   1.000
      2        3  static_synapse   -2.000   1.000
      2        6  static_synapse   -2.000   1.000
      4        4  static_synapse   -2.000   1.000
      4        8  static_synapse   -2.000   1.000
      5        5  static_synapse   -2.000   1.000
      7        7  static_synapse   -2.000   1.000
      1        1    stdp_synapse    3.000   1.000
      2        2    stdp_synapse    3.000   1.000
      2        3    stdp_synapse    3.000   1.000
      2        6    stdp_synapse    3.000   1.000
      4        4    stdp_synapse    3.000   1.000
      4        8    stdp_synapse    3.000   1.000
      5        5    stdp_synapse    3.000   1.000
      7        7    stdp_synapse    3.000   1.000
